# Fine-Tuning and Evaluation (Merged) Notebook
- Cycle 2

### Fine Tuning Section
- Using Unsloth
- Code is a slightly modified version of the code provided in the Unsloth Github Repository

##### Installation and GDrive Setup

In [ ]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

from google.colab import drive
drive.mount('/content/drive')

##### Fine-Tuning Setup

In [ ]:
from unsloth import FastLanguageModel
import torch

## Setup
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

## Load Llama 3.1
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

## Default Unsloth Settings
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## Data Loading and Alpaca Prompt Setup (Default Prompt Setup for Llama 3.1)
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset_location = "/content/drive/MyDrive/Fine-Tuning Llama 3.1 For Test Cases/fine_tuning_cycle_1.csv"
dataset = load_dataset('csv', data_files=dataset_location, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

## Trainer Setup
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

##### Training Proper

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

trainer_stats = trainer.train()

# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

##### Initial Testing

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Generate a test case for the following UI Element:", # instruction #
        "Link Element : ", # input
        "", # output - leave this blank for generation!
    ),

    alpaca_prompt.format(
        "Generate a test case for the following UI Element:", # instruction
        "Button Element : ", # input
        "", # output - leave this blank for generation!
    ),

    alpaca_prompt.format(
        "Generate a test case for the following UI Element:", # instruction #
        "Header Element : ", #
        "", # output - leave this blank for generation!
    ),

    alpaca_prompt.format(
        "Generate a test case for the following UI Element:", # instruction #
        "Input Element : ", #
        "", # output - leave this blank for generation!
    )

], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
repeated_outputs = model.generate(**inputs, max_new_tokens=64, use_cache = True)

tokenizer.batch_decode(outputs)

##### Saving Model

In [ ]:
## LORA Saving
if False:
    model.save_pretrained("lora_model")  # Local saving
    tokenizer.save_pretrained("lora_model")
    # model.push_to_hub("your_name/lora_model", token = "...") # Online saving
    # tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

## GGUF Saving 
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

### Evaluation Section
- Evaluating perplexity and hallucination (Automated evaluation metrics)
- Perplexity is calculated using Huggingface
- Hallucination is calculated using SelfCheckGPT score/s

In [ ]:
! pip install selfcheckgpt
! pip install spacy

##### Perplexity

In [ ]:
# Imports
from transformers import LlamaTokenizer, LlamaForCausalLM
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
from tqdm import tqdm

# Evaluate Perplexity using Torch
def compute_perplexity(model, dataset, tokenizer):
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
    dataloader = DataLoader(dataset, collate_fn=data_collator, batch_size=8)
    model.eval()
    total_loss = 0
    total_length = 0
    
    with torch.no_grad():
        for batch in tqdm(dataloader):
            inputs = batch["input_ids"]
            outputs = model(inputs, labels=inputs)
            loss = outputs.loss
            total_loss += loss.item() * inputs.size(1)
            total_length += inputs.size(1)
    
    perplexity = torch.exp(total_loss / total_length)
    return perplexity.item()

# # Load Model
# model_location = ""
# # Directory of pytorch .bin files
# tokenizer = LlamaTokenizer.from_pretrained(model_location)
# model = LlamaForCausalLM.from_pretrained(model_location)

# Perplexity Evaluation Dataset
dataset_location = ""
eval_dataset = load_dataset(dataset_location)

perplexity = compute_perplexity(model, eval_dataset, tokenizer)
print(f"Perplexity: {perplexity}")

##### Hallucination with SelfCheckGPT

In [ ]:
import torch
import spacy
from selfcheckgpt.modeling_selfcheck import SelfCheckLLMPrompt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
llm_model = "mistralai/Mistral-7B-Instruct-v0.2"
selfcheck_prompt = SelfCheckLLMPrompt(llm_model, device)

## Manually entered in responses
# TODO : Properly Format Outputs
# Response
sentences = outputs
# Response 2 / Samples
samples = repeated_outputs

sent_scores_prompt = selfcheck_prompt.predict(
    sentences = sentences,                         
    sampled_passages = samples,
    verbose = True,
)
print(sent_scores_prompt)